In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier

from csv import writer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score,classification_report, confusion_matrix
from sklearn import tree
import matplotlib.pyplot as plt
import time

C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1857202733.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:

def GetAttributeMapping(data,grp=None,grp_gap=20,map_type=1):
    #print('in mapping')
    #print(data)
    #for col in data.columns[:7]:
    #        print(col,data[col].unique())
    mapping = {}
    mapping['NotA'] = -1
    mapping["NA"] = -1
    mapping[0] = 0
    #mapping[1] = 1
    mapping['Yes'] = 1
    mapping['No'] = 0
    if(map_type==1):
        for col in data.columns[:7]:
            #print(col,data[col].unique())
            col_un = data[col].unique()
            cnt = 1
            for val in col_un:
                if(val != 'NotA'):
                    mapping[val] = cnt
                    cnt = cnt + 1
        #print(mapping)
        return mapping
    elif(map_type==2):
        for col in data.columns[2:6]:
            #print(col,it_full[col].unique())
            col_un = data[col].unique()
            cnt = 1
            for val in col_un:
                if(val != 'NotA'):
                    mapping[val] = cnt
                    cnt = cnt + 1
        
        for g in grp:
            grp_num = 1
            for member in grp[g]:
                mem_num = 1
                for val in member:
                    mapping[val] = grp_num*grp_gap+mem_num
                    mem_num = mem_num + 1
                grp_num = grp_num + 1
        #print(mapping)
        return mapping
  

In [3]:

designation_grp = [['officer'],
                   ['prof','adj_prof','vis_prof'],
                   ['stu']
                  ]

type_grp = [['asgn','quiz'],
            ['off_rec','dept_bud','proj'],
            ['std_mat'],
            ['attdn','stu_rec'],
            ['q_pr','grade_book']
           ]

attr_grp={}
attr_grp['Designation'] = designation_grp
attr_grp['Type'] = type_grp

In [4]:
def same_conditions(col1,col2):
    if ((col1 == -1) or (col2 == -1)):
        return 2
    elif (col1==col2):
        return 1
    else:
        return 0

def chk_nota(col):
    if(col==-1):
        return 1
    else:
        return 0

In [5]:
def GetPreparedData(train_data,test_data,prep_type=4):
    #print('in prepare data')
    data = pd.concat([train_data,test_data],axis=0)
    if(prep_type==1):#Naive
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        #print(mapping)
        data_encoded = data.replace(mapping)
    elif(prep_type==2):#Columns for same attribute values in subject and object
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded['sameDeg'] = data_encoded.apply(lambda x: same_conditions(x['Degree'], x['Degree.1']), axis=1)
        data_encoded['sameYr'] = data_encoded.apply(lambda x: same_conditions(x['Year'], x['Year.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Degree',axis=1)
        data_encoded = data_encoded.drop('Degree.1',axis=1)
        data_encoded = data_encoded.drop('Year',axis=1)
        data_encoded = data_encoded.drop('Year.1',axis=1)
    elif(prep_type==3):#Grouping of attributes
        map_type = 2
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
    elif(prep_type==4):#Grouping of attributes + Columns for same attribute values in subject and object
        map_type = 2
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded['sameDeg'] = data_encoded.apply(lambda x: same_conditions(x['Degree'], x['Degree.1']), axis=1)
        data_encoded['sameYr'] = data_encoded.apply(lambda x: same_conditions(x['Year'], x['Year.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Degree',axis=1)
        data_encoded = data_encoded.drop('Degree.1',axis=1)
        data_encoded = data_encoded.drop('Year',axis=1)
        data_encoded = data_encoded.drop('Year.1',axis=1)
    elif(prep_type==5):#Naive+NACol
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['Year_NA'] = data_encoded.apply(lambda x: chk_nota(x['Year']), axis=1)
        data_encoded['Year.1_NA'] = data_encoded.apply(lambda x: chk_nota(x['Year.1']), axis=1)
        data_encoded['Degree_NA'] = data_encoded.apply(lambda x: chk_nota(x['Degree']), axis=1)
        data_encoded['Degree.1_NA'] = data_encoded.apply(lambda x: chk_nota(x['Degree.1']), axis=1)
    
    X = data_encoded.loc[:, data_encoded.columns != 'Access']
    y = data_encoded.loc[:, data_encoded.columns == 'Access']
    
    X_train = X.head(train_data.shape[0])
    X_test = X.tail(test_data.shape[0])
    y_train = y.head(train_data.shape[0])
    y_test = y.tail(test_data.shape[0])
    #print(X)
    return X_train, X_test, y_train, y_test

In [6]:

models_dict = {
    #"SupportVectorMachine": SVC(kernel="linear"),
    #"LogisticRegression": LogisticRegression(solver="saga", n_jobs=-1),
    "ArtificalNeuralNetwork": MLPClassifier(hidden_layer_sizes=30, max_iter=1500),
    "DecisionTree": DecisionTreeClassifier(),
    #"ExtraTree": ExtraTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=-1),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="error", n_jobs=-1),
    #"LightGBM": LGBMClassifier(n_estimators=100, n_jobs=-1),
    #"AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=1.0),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),
}

In [7]:


#DESIGNATION	Project_name	Department	Resource	Project_Name	Department
fn=['S-Designation', 'S-Project', 'S-Dept', 'O-Res','O-Project', 'O-Dept']
cn=['AccessDenied', 'AccessGranted']

for (name, clf) in models_dict.items():
        train_data = pd.read_csv('University1/university_saptarshi_train.csv')
        test_data = pd.read_csv('change_University1.csv')
        test1_data = pd.read_csv('change2_University1.csv')
        X_train, X_test, y_train, y_test = GetPreparedData(train_data,test_data,prep_type=3)
        X_train1, X_test1, y_train1, y_test1 = GetPreparedData(train_data,test1_data,prep_type=3)
        clf.fit(X_train.to_numpy(), y_train.to_numpy())
        clf.fit(X_train1.to_numpy(), y_train1.to_numpy())
        pred = clf.predict(X_test.to_numpy())
        pred1 = clf.predict(X_test1.to_numpy())
        
        print(confusion_matrix(y_test, pred))
        print(classification_report(y_test, pred))
        
        print(confusion_matrix(y_test1, pred1))
        print(classification_report(y_test1, pred1))
        
        #write these to the file
        
        with open("result_univ1_avc.txt", 'a') as f:
            f.write(name + "\n")
            f.write("Test Data\n")
            f.write(str(confusion_matrix(y_test, pred))) # Convert ndarray to string
            f.write(str(classification_report(y_test, pred))) # Convert ndarray to string
            f.write("Test1 Data\n")
            f.write(str(confusion_matrix(y_test1, pred1))) # Convert ndarray to string
            f.write(str(classification_report(y_test1, pred1))) # Convert ndarray to string
            f.write("\n\n")
        
        

C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y

[[280   0]
 [136 204]]
              precision    recall  f1-score   support

           0       0.67      1.00      0.80       280
           1       1.00      0.60      0.75       340

    accuracy                           0.78       620
   macro avg       0.84      0.80      0.78       620
weighted avg       0.85      0.78      0.77       620

[[120  12]
 [ 48 210]]
              precision    recall  f1-score   support

           0       0.71      0.91      0.80       132
           1       0.95      0.81      0.88       258

    accuracy                           0.85       390
   macro avg       0.83      0.86      0.84       390
weighted avg       0.87      0.85      0.85       390

[[264  16]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       280
           1       0.96      1.00      0.98       340

    accuracy                           0.97       620
   macro avg       0.98      0.97      0.97       620
weigh

C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was e

[[264  16]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       280
           1       0.96      1.00      0.98       340

    accuracy                           0.97       620
   macro avg       0.98      0.97      0.97       620
weighted avg       0.98      0.97      0.97       620

[[129   3]
 [ 13 245]]
              precision    recall  f1-score   support

           0       0.91      0.98      0.94       132
           1       0.99      0.95      0.97       258

    accuracy                           0.96       390
   macro avg       0.95      0.96      0.95       390
weighted avg       0.96      0.96      0.96       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please cha

[[264  16]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       280
           1       0.96      1.00      0.98       340

    accuracy                           0.97       620
   macro avg       0.98      0.97      0.97       620
weighted avg       0.98      0.97      0.97       620

[[127   5]
 [  8 250]]
              precision    recall  f1-score   support

           0       0.94      0.96      0.95       132
           1       0.98      0.97      0.97       258

    accuracy                           0.97       390
   macro avg       0.96      0.97      0.96       390
weighted avg       0.97      0.97      0.97       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)


[[264  16]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      0.94      0.97       280
           1       0.96      1.00      0.98       340

    accuracy                           0.97       620
   macro avg       0.98      0.97      0.97       620
weighted avg       0.98      0.97      0.97       620

[[130   2]
 [  4 254]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       132
           1       0.99      0.98      0.99       258

    accuracy                           0.98       390
   macro avg       0.98      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:25: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expe

[[264  16]
 [ 36 304]]
              precision    recall  f1-score   support

           0       0.88      0.94      0.91       280
           1       0.95      0.89      0.92       340

    accuracy                           0.92       620
   macro avg       0.92      0.92      0.92       620
weighted avg       0.92      0.92      0.92       620

[[132   0]
 [  8 250]]
              precision    recall  f1-score   support

           0       0.94      1.00      0.97       132
           1       1.00      0.97      0.98       258

    accuracy                           0.98       390
   macro avg       0.97      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390



In [8]:


#DESIGNATION	Project_name	Department	Resource	Project_Name	Department
fn=['S-Designation', 'S-Project', 'S-Dept', 'O-Res','O-Project', 'O-Dept']
cn=['AccessDenied', 'AccessGranted']

for (name, clf) in models_dict.items():
        train_data = pd.read_csv('University1/university_saptarshi_train.csv')
        test_data = pd.read_csv('change_University1.csv')
        test1_data = pd.read_csv('change2_University1.csv')
        X_train, X_test, y_train, y_test = GetPreparedData(train_data,test_data,prep_type=4)
        X_train1, X_test1, y_train1, y_test1 = GetPreparedData(train_data,test1_data,prep_type=4)
        clf.fit(X_train.to_numpy(), y_train.to_numpy())
        clf.fit(X_train1.to_numpy(), y_train1.to_numpy())
        pred = clf.predict(X_test.to_numpy())
        pred1 = clf.predict(X_test1.to_numpy())
        
        print(confusion_matrix(y_test, pred))
        print(classification_report(y_test, pred))
        
        print(confusion_matrix(y_test1, pred1))
        print(classification_report(y_test1, pred1))
        
        #write these to the file
        
        with open("result_univ1_avc_arfe.txt", 'a') as f:
            f.write(name + "\n")
            f.write("Test Data\n")
            f.write(str(confusion_matrix(y_test, pred))) # Convert ndarray to string
            f.write(str(classification_report(y_test, pred))) # Convert ndarray to string
            f.write("Test1 Data\n")
            f.write(str(confusion_matrix(y_test1, pred1))) # Convert ndarray to string
            f.write(str(classification_report(y_test1, pred1))) # Convert ndarray to string
            f.write("\n\n")
        
        

C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:1101: DataConversionWarning: A column-vector y

[[280   0]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       280
           1       1.00      1.00      1.00       340

    accuracy                           1.00       620
   macro avg       1.00      1.00      1.00       620
weighted avg       1.00      1.00      1.00       620

[[130   2]
 [  0 258]]
              precision    recall  f1-score   support

           0       1.00      0.98      0.99       132
           1       0.99      1.00      1.00       258

    accuracy                           0.99       390
   macro avg       1.00      0.99      0.99       390
weighted avg       0.99      0.99      0.99       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)


[[280   0]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       280
           1       1.00      1.00      1.00       340

    accuracy                           1.00       620
   macro avg       1.00      1.00      1.00       620
weighted avg       1.00      1.00      1.00       620

[[130   2]
 [  4 254]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       132
           1       0.99      0.98      0.99       258

    accuracy                           0.98       390
   macro avg       0.98      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was e

...

[[280   0]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       280
           1       1.00      1.00      1.00       340

    accuracy                           1.00       620
   macro avg       1.00      1.00      1.00       620
weighted avg       1.00      1.00      1.00       620

[[130   2]
 [  4 254]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       132
           1       0.99      0.98      0.99       258

    accuracy                           0.98       390
   macro avg       0.98      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)


[[280   0]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       280
           1       1.00      1.00      1.00       340

    accuracy                           1.00       620
   macro avg       1.00      1.00      1.00       620
weighted avg       1.00      1.00      1.00       620

[[130   2]
 [  4 254]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       132
           1       0.99      0.98      0.99       258

    accuracy                           0.98       390
   macro avg       0.98      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390



C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Temp\ipykernel_16080\1355176860.py:29: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_encoded = data.replace(mapping)
C:\Users\tipsh\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\preprocessing\_label.py:114: DataConversionWarning: A column-vector y was passed when a

[[280   0]
 [  0 340]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00       280
           1       1.00      1.00      1.00       340

    accuracy                           1.00       620
   macro avg       1.00      1.00      1.00       620
weighted avg       1.00      1.00      1.00       620

[[130   2]
 [  4 254]]
              precision    recall  f1-score   support

           0       0.97      0.98      0.98       132
           1       0.99      0.98      0.99       258

    accuracy                           0.98       390
   macro avg       0.98      0.98      0.98       390
weighted avg       0.98      0.98      0.98       390

